In [1]:
import numpy as np
import pandas as pd
import sympy as sp
from fractions import Fraction

In [2]:
NUM_PLAYERS = 3
PLAYERS = list(range(1, NUM_PLAYERS+1))
# all possible (giver, receiver) pairs
OUTCOME_PROB = Fraction(1, NUM_PLAYERS * (NUM_PLAYERS-1))  # 1/6
CANDIDATE_MOVES = [(g, r) for idx, g in enumerate(PLAYERS) for r in PLAYERS if g != r]
LOSER_IDX = 0

In [3]:
def applyMove(initState, move):
    g, r = move
    initWealth = list(initState)
    transferAmt = min(initWealth[g-1], initWealth[r-1])  # 0-indexed
    initWealth[g-1] -= transferAmt  # give
    initWealth[r-1] += transferAmt  # receive
    return tuple(initWealth)


def getChildStates(initState):
    """
    Applies all possible moves to the initial state.
    Returns the child states.
    """
    return [applyMove(initState, move) for move in CANDIDATE_MOVES]

def getCoef(initState):
    """
    Returns the coefficients for a first-step analysis equation,
    where the hitting probability of the initial state is the subject.
    """
    # assign all states the individual outcome probability
    childStates = getChildStates(initState)
    stateCoefs = {s:OUTCOME_PROB for s in childStates}
    const = 0
    
    for state in stateCoefs.copy():
        # prune the 0 hitting prob states
        if zeroHittingProb(state):
            stateCoefs.pop(state)
        # convert the hitting prob 1 states to constants
        elif alreadyHit(state):
            const += stateCoefs[state]
            stateCoefs.pop(state)
        # optimsation 1: convert equal wealth states to constants
        elif equalWealth(state):
            const += stateCoefs[state] / NUM_PLAYERS
            stateCoefs.pop(state)
    
    # optimization 2: combine symmetrical states
    copy = list(stateCoefs.copy().keys())
    n = len(copy)
    for i in range(n):
        for j in range(i+1, n):
            s1, s2 = copy[i], copy[j]
            if (symmetricalStates(s1, s2)):
                combinedProb = stateCoefs.pop(s1) + stateCoefs.pop(s2)
                stateCoefs[standardiseState(s1)] = combinedProb
    
    return standardiseStates(stateCoefs), const

def zeroHittingProb(state):
    """
    Returns true if the given state cannot reach the state of Player 1 being the loser, false otherwise.
    
    """
    return (0 in state) and (state[LOSER_IDX] > 0)

def alreadyHit(state):
    """
    Returns true if the given state reaches the state of Player 1 being the loser wp1, false otherwise.
    
    """
    return state[LOSER_IDX] == 0

def symmetricalStates(s1, s2):
    return s1[0] == s2[0] and set(s1[1:]) == set(s2[1:])

def standardiseState(s):
    # convention: keep the smaller value at the front
    return s if s[1] <= s[2] else (s[0], s[2], s[1])

def standardiseStates(stateCoefs):
    return {standardiseState(s):p for s, p in stateCoefs.items()}
    
def equalWealth(s):
    return s[0] == s[1] and s[1] == s[2]

In [4]:
from sympy import *
from sympy import nsolve
from sympy.solvers import solve
from ast import literal_eval as make_tuple  # for parsing state as a tuple

In [5]:
getCoef((2,2,2))

({}, Fraction(1, 3))

In [34]:
def getEquation(initState, stateCoefs, const):
    """
    Converts the dictionary of state:coefficient pairs and constant
    into an equality of the form: h(initState) = coefs * h(states) + const
    """
    # equation will be in the form of human readable equalities (like how we normally write down)
    symbols = [stateToSymbol(s) for s in stateCoefs]
    eqn = Eq(stateToSymbol(initState), getExpr(stateCoefs, const))
    return eqn, symbols
    
def stateToSymbol(state):
    """
    Converts the state (triple) to a Symbol object.
    """
    return Symbol(f'h{state}')

def symbolToState(symbol):
    """
    Converts the Symbol object to a state (triple).
    """
    return make_tuple(str(symbol)[1:])

def getExpr(stateCoefs, const):
    """
    Converts the dictionary of state:coefficient pairs and constant
    into an expression.
    """
    expr = const
    for s in stateCoefs:
        expr = Add(expr, stateCoefs[s] * stateToSymbol(s))
    return expr

def formatEquation(eqn):
    """
    Returns the formatted string for the equation.
    """
    return f'{eqn.lhs} = {eqn.rhs}'

def formatSolutions(sol):
    """
    Returns the solutions as a formatted list (states converted back to triples)
    """
    return [(symbolToState(s), p) for s, p in sol.items()]

In [7]:
class LoserAnalysis():
    def __init__(self, initState):
        self.initState = standardiseState(initState)
        self.generatedStates = set([initState])
        self.equations = []
        self.symbols = set([stateToSymbol(self.initState)]) # or can encode all generated states at once in the end
    
    def getEquations(self):
        """
        Returns a list of equations (as Equality objects) generated during the first step analysis.
        """
        self.recurseGenerateEqns(self.initState)
        return self.equations
        
    def recurseGenerateEqns(self, initState):
        """
        Recursively generates the first step analysis equations,
        and records the new symbols and equations.
        """
        childStatesProb, const = getCoef(initState)  # state:prob pairs, under unified representation
        newStates = set(childStatesProb).difference(self.generatedStates)
        
        # compute for the current state
        # do this before the recursion to close off the loop
        eqn, symbols = getEquation(initState, childStatesProb, const)
        self.equations.append(eqn)

        # base case: no new states
        if not newStates:
            return

        self.symbols = self.symbols.union(symbols)
        self.generatedStates = self.generatedStates.union(newStates)
        
        # recurse
        for state in newStates:
            self.recurseGenerateEqns(state)
    
    def solveEqns(self):
        """
        Returns the hitting probabilities of the states involved in the analysis.
        """
        self.getEquations()
        return solve(self.equations, self.symbols)
    
    # numerical approx
#     def nsolveEqns(self):
#         self.getEquations()
#         exprs = 
#         return nsolve(self.equations, self.symbols, np.repeat(1, len(self.symbols)))
    
    def exportEqns(self, filename):
        """
        Exports the first step analysis equations to a text file, 
        using the provided filename.
        """
        self.getEquations()
        fp = open(filename, 'w')
        for e in self.equations:
            fp.write(formatEquation(e) + '\n')
        fp.close()

In [14]:
LoserAnalysis((1,2,3)).solveEqns()

{h(1, 1, 4): 44/93,
 h(1, 2, 3): 113/217,
 h(2, 1, 3): 69/217,
 h(3, 1, 2): 5/31,
 h(4, 1, 1): 5/93}

In [9]:
LoserAnalysis((5,9,3)).solveEqns()

{h(1, 2, 14): 1529250476843789301594/2626941360963688147249,
 h(1, 4, 12): 1587632244934798889985/2626941360963688147249,
 h(1, 6, 10): 1640810152833367160839/2626941360963688147249,
 h(1, 8, 8): 1360397809099802318197/2626941360963688147249,
 h(10, 1, 6): 247143895297849989524/2626941360963688147249,
 h(10, 2, 5): 278213826966303319797/2626941360963688147249,
 h(10, 3, 4): 21373553795588457943/202072412381822165173,
 h(11, 2, 4): 194380219368060771093/2626941360963688147249,
 h(12, 1, 4): 12237802262295428568/202072412381822165173,
 h(12, 2, 3): 126896581159039742685/2626941360963688147249,
 h(13, 2, 2): 64793406456020257031/2626941360963688147249,
 h(14, 1, 2): 44776967173172165683/2626941360963688147249,
 h(2, 1, 14): 1052913916946726679972/2626941360963688147249,
 h(2, 2, 13): 1281073977253833945109/2626941360963688147249,
 h(2, 3, 12): 1438371019815848745644/2626941360963688147249,
 h(2, 4, 11): 1473141918765765981571/2626941360963688147249,
 h(2, 5, 10): 1495573108152538186574/26

In [10]:
LoserAnalysis((2,2,2)).solveEqns()

{h(2, 2, 2): 1/3}

In [28]:
MAX_AMT = 10
MIN_AMT = 1
FILENAME = 'player1LoseProbs_10.csv'

In [36]:
# aggregate to text file

stateProbs = dict()  # states as symbols : hitting prob

for i in range(MIN_AMT, MAX_AMT+1):
    for j in range(MIN_AMT, MAX_AMT+1):
        for k in range(MIN_AMT, MAX_AMT+1):
            initState = (i, j, k)
            sols = LoserAnalysis(initState).solveEqns()
            if sols:
                newStates = set(sols).difference(stateProbs)
                for s in newStates:
                    stateProbs[s] = sols[s]

# sort by state
stateProbs = dict(sorted(formatSolutions(stateProbs)))

stateProbs

{(1, 1, 1): 1/3,
 (1, 1, 2): 3/7,
 (1, 1, 3): 19/42,
 (1, 1, 4): 44/93,
 (1, 1, 5): 631/1308,
 (1, 1, 6): 22376/46095,
 (1, 1, 8): 23935027/48690978,
 (1, 1, 10): 55410683846/111972079949,
 (1, 1, 12): 273248782365117911/550686146155668984,
 (1, 2, 2): 3/7,
 (1, 2, 3): 113/217,
 (1, 2, 4): 60/109,
 (1, 2, 5): 59786/107555,
 (1, 2, 6): 8298/15019,
 (1, 2, 7): 32214606/56806141,
 (1, 2, 8): 4052550/7036351,
 (1, 2, 9): 9520691348078/16459895752503,
 (1, 2, 10): 124031295624/213378126031,
 (1, 2, 11): 373892245050675355/642467170514947148,
 (1, 3, 3): 307/654,
 (1, 3, 4): 1207/2195,
 (1, 3, 5): 52181/90114,
 (1, 3, 6): 4770930/8115163,
 (1, 3, 8): 1400282283425/2351413678929,
 (1, 3, 10): 329163154048825171/550686146155668984,
 (1, 4, 4): 7379/15019,
 (1, 4, 5): 27650603/48690978,
 (1, 4, 6): 602591/1005193,
 (1, 4, 7): 1426448389150/2351413678929,
 (1, 4, 8): 130877374629/213378126031,
 (1, 4, 9): 9947088885861389/16196651357519676,
 (1, 5, 5): 3522116/7036351,
 (1, 5, 6): 1355257729024/

In [41]:
# export to CSV
import csv

fp = open(FILENAME, 'w')
writer = csv.writer(fp)
writer.writerow(['Initial state', 'P(Loser = Player 1)'])
writer.writerows(stateProbs.items())
fp.close()